In [1]:
import os
os.environ['TF_DEVICE_MIN_SYS_MEMORY_IN_MB'] = '128' 

os.environ['PYTORCH_CUDA_ALLOC_CONF'] = 'expandable_segments:True'

In [2]:
from huggingface_hub import notebook_login

notebook_login(new_session=False)

User is already logged in.


In [3]:
language = 'bn'
model_str = 'whisper-tiny'

In [4]:
from datasets import load_dataset, IterableDatasetDict, DatasetDict

common_voice = IterableDatasetDict()


common_voice["train"] = load_dataset("mozilla-foundation/common_voice_11_0", language, split="train", use_auth_token=True, streaming=True)
common_voice["validation"] = load_dataset("mozilla-foundation/common_voice_11_0", language, split="validation", use_auth_token=True, streaming=True)
common_voice["test"] = load_dataset("mozilla-foundation/common_voice_11_0", language, split="test", use_auth_token=True, streaming=True)

print(common_voice)

/home/ahanaf/Torch/torch2_2/lib/python3.11/site-packages/datasets/load.py:2516: FutureWarning: 'use_auth_token' was deprecated in favor of 'token' in version 2.14.0 and will be removed in 3.0.0.
You can remove this warning by passing 'token=<use_auth_token>' instead.
  warnings.warn(
/home/ahanaf/Torch/torch2_2/lib/python3.11/site-packages/datasets/load.py:1461: FutureWarning: The repository for mozilla-foundation/common_voice_11_0 contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/mozilla-foundation/common_voice_11_0
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this dataset from the next major release of `datasets`.
  warnings.warn(
/home/ahanaf/Torch/torch2_2/lib/python3.11/site-packages/datasets/load.py:2516: FutureWarning: 'use_auth_token' was deprecated in favor of 'token' in version 2.14.0 and

IterableDatasetDict({
    train: IterableDataset({
        features: ['client_id', 'path', 'audio', 'sentence', 'up_votes', 'down_votes', 'age', 'gender', 'accent', 'locale', 'segment'],
        n_shards: 1
    })
    validation: IterableDataset({
        features: ['client_id', 'path', 'audio', 'sentence', 'up_votes', 'down_votes', 'age', 'gender', 'accent', 'locale', 'segment'],
        n_shards: 1
    })
    test: IterableDataset({
        features: ['client_id', 'path', 'audio', 'sentence', 'up_votes', 'down_votes', 'age', 'gender', 'accent', 'locale', 'segment'],
        n_shards: 1
    })
})


In [5]:
common_voice["test"]

IterableDataset({
    features: ['client_id', 'path', 'audio', 'sentence', 'up_votes', 'down_votes', 'age', 'gender', 'accent', 'locale', 'segment'],
    n_shards: 1
})

In [6]:
common_voice = common_voice.remove_columns(["accent", "age", "client_id", "down_votes", "gender", "locale", "path", "segment", "up_votes"])

In [7]:
from transformers import WhisperFeatureExtractor
from transformers import WhisperTokenizer
from transformers import WhisperProcessor


feature_extractor = WhisperFeatureExtractor.from_pretrained(f"openai/{model_str}")
tokenizer = WhisperTokenizer.from_pretrained(f"openai/{model_str}", language='Bengali', task="transcribe")
processor = WhisperProcessor.from_pretrained(f"openai/{model_str}", language="Bengali", task="transcribe")

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [8]:
# for x in common_voice['train']:
#     arr = x['audio']['array']
#     data_sample = x
#     print(arr.min(), arr.max())
#     break
    

In [9]:
# input_str = data_sample["sentence"]
# labels = tokenizer(input_str).input_ids
# decoded_with_special = tokenizer.decode(labels, skip_special_tokens=False)
# decoded_str = tokenizer.decode(labels, skip_special_tokens=True)

# print(f"Input:                 {input_str}")
# print(f"Decoded w/ special:    {decoded_with_special}")
# print(f"Decoded w/out special: {decoded_str}")
# print(f"Are equal:             {input_str == decoded_str}")

In [10]:
print(common_voice["train"])

IterableDataset({
    features: ['audio', 'sentence'],
    n_shards: 1
})


In [11]:
from datasets import Audio

common_voice = common_voice.cast_column("audio", Audio(sampling_rate=16000))

In [12]:
# for x in common_voice["train"]:
#     print(x)
#     break

In [13]:
def prepare_dataset(batch):
    # load and resample audio data from 48 to 16kHz
    # print(batch)
    audio = batch["audio"]

    # compute log-Mel input features from input audio array 
    batch["input_features"] = feature_extractor(audio["array"], sampling_rate=audio["sampling_rate"]).input_features[0]

    # encode target text to label ids 
    batch["labels"] = tokenizer(batch["sentence"]).input_ids
    return batch

In [14]:
mapped_common_voice = common_voice.map(prepare_dataset)

In [15]:
mapped_common_voice['train']

IterableDataset({
    features: Unknown,
    n_shards: 1
})

In [16]:
# for x in mapped_common_voice['train']:
#     print(x)
#     break

In [17]:
import torch

from dataclasses import dataclass
from typing import Any, Dict, List, Union

@dataclass
class DataCollatorSpeechSeq2SeqWithPadding:
    processor: Any

    def __call__(self, features: List[Dict[str, Union[List[int], torch.Tensor]]]) -> Dict[str, torch.Tensor]:
        # split inputs and labels since they have to be of different lengths and need different padding methods
        # first treat the audio inputs by simply returning torch tensors
        input_features = [{"input_features": feature["input_features"]} for feature in features]
        batch = self.processor.feature_extractor.pad(input_features, return_tensors="pt")

        # get the tokenized label sequences
        label_features = [{"input_ids": feature["labels"]} for feature in features]
        # pad the labels to max length
        labels_batch = self.processor.tokenizer.pad(label_features, return_tensors="pt")

        # replace padding with -100 to ignore loss correctly
        labels = labels_batch["input_ids"].masked_fill(labels_batch.attention_mask.ne(1), -100)

        # if bos token is appended in previous tokenization step,
        # cut bos token here as it's append later anyways
        if (labels[:, 0] == self.processor.tokenizer.bos_token_id).all().cpu().item():
            labels = labels[:, 1:]

        batch["labels"] = labels

        return batch

In [18]:
data_collator = DataCollatorSpeechSeq2SeqWithPadding(processor=processor)

In [19]:
import evaluate

metric = evaluate.load("wer")

In [20]:
def compute_metrics(pred):
    pred_ids = pred.predictions
    label_ids = pred.label_ids

    # replace -100 with the pad_token_id
    label_ids[label_ids == -100] = tokenizer.pad_token_id

    # we do not want to group tokens when computing the metrics
    pred_str = tokenizer.batch_decode(pred_ids, skip_special_tokens=True)
    label_str = tokenizer.batch_decode(label_ids, skip_special_tokens=True)

    wer = 100 * metric.compute(predictions=pred_str, references=label_str)

    return {"wer": wer}

In [21]:
from transformers import WhisperForConditionalGeneration

model = WhisperForConditionalGeneration.from_pretrained(f"openai/{model_str}")

In [22]:
model.generation_config.language = "bengali"
model.generation_config.task = "transcribe"

model.generation_config.forced_decoder_ids = None

In [23]:
from transformers import Seq2SeqTrainingArguments

training_args = Seq2SeqTrainingArguments(
    output_dir=f"./{model_str}-{language}2",  # change to a repo name of your choice
    per_device_train_batch_size=16,
    gradient_accumulation_steps=1,  # increase by 2x for every 2x decrease in batch size
    learning_rate=1e-5,
    warmup_steps=500,
    max_steps=14000,
    gradient_checkpointing=True,
    fp16=True,
    evaluation_strategy="steps",
    per_device_eval_batch_size=16,
    predict_with_generate=True,
    generation_max_length=400,
    save_steps=2000,
    eval_steps=2000,
    logging_steps=25,
    report_to=["tensorboard"],
    load_best_model_at_end=True,
    metric_for_best_model="wer",
    greater_is_better=False,
    # push_to_hub=True,
    dataloader_prefetch_factor=128,
    dataloader_num_workers=16,
    auto_find_batch_size=True,
    # hub_private_repo=
    resume_from_checkpoint=True,
)

In [24]:
from transformers import Seq2SeqTrainer

trainer = Seq2SeqTrainer(
    args=training_args,
    model=model,
    train_dataset=mapped_common_voice["train"],
    eval_dataset=mapped_common_voice["test"],
    data_collator=data_collator,
    compute_metrics=compute_metrics,
    tokenizer=processor.feature_extractor,
)

/home/ahanaf/Torch/torch2_2/lib/python3.11/site-packages/accelerate/accelerator.py:436: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(


In [26]:
trainer.train(resume_from_checkpoint=True)

  0%|          | 0/14000 [00:00<?, ?it/s]

Too many dataloader workers: 16 (max is dataset.n_shards=1). Stopping 15 dataloader workers.
Reading metadata...: 16777it [00:03, 5571.44it/s]
/home/ahanaf/Torch/torch2_2/lib/python3.11/site-packages/torch/utils/checkpoint.py:460: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
`use_cache = True` is incompatible with gradient checkpointing. Setting `use_cache = False`...


{'loss': 3.1569, 'grad_norm': 87.94834899902344, 'learning_rate': 4.2000000000000006e-07, 'epoch': 0.0}
{'loss': 2.7364, 'grad_norm': 28.024662017822266, 'learning_rate': 9.200000000000001e-07, 'epoch': 0.0}
{'loss': 2.269, 'grad_norm': 13.538902282714844, 'learning_rate': 1.42e-06, 'epoch': 0.01}
{'loss': 1.9094, 'grad_norm': 12.83574104309082, 'learning_rate': 1.9200000000000003e-06, 'epoch': 0.01}
{'loss': 1.67, 'grad_norm': 8.416497230529785, 'learning_rate': 2.42e-06, 'epoch': 0.01}
{'loss': 1.5258, 'grad_norm': 13.104721069335938, 'learning_rate': 2.92e-06, 'epoch': 0.01}
{'loss': 1.4346, 'grad_norm': 11.164371490478516, 'learning_rate': 3.4200000000000007e-06, 'epoch': 0.01}
{'loss': 1.3795, 'grad_norm': 8.804588317871094, 'learning_rate': 3.920000000000001e-06, 'epoch': 0.01}
{'loss': 1.3405, 'grad_norm': 8.74685001373291, 'learning_rate': 4.42e-06, 'epoch': 0.02}
{'loss': 1.3196, 'grad_norm': 10.954477310180664, 'learning_rate': 4.92e-06, 'epoch': 0.02}
{'loss': 1.2939, 'grad_

Too many dataloader workers: 16 (max is dataset.n_shards=1). Stopping 15 dataloader workers.
Reading metadata...: 16777it [00:03, 4805.80it/s]


{'loss': 0.3963, 'grad_norm': 7.380018711090088, 'learning_rate': 9.595555555555556e-06, 'epoch': 1.0}
{'loss': 0.3506, 'grad_norm': 6.660755157470703, 'learning_rate': 9.577037037037039e-06, 'epoch': 1.0}
{'loss': 0.3492, 'grad_norm': 8.48245620727539, 'learning_rate': 9.55851851851852e-06, 'epoch': 1.0}
{'loss': 0.3446, 'grad_norm': 9.087663650512695, 'learning_rate': 9.54e-06, 'epoch': 1.01}
{'loss': 0.3484, 'grad_norm': 8.031760215759277, 'learning_rate': 9.521481481481483e-06, 'epoch': 1.01}
{'loss': 0.3368, 'grad_norm': 6.4939985275268555, 'learning_rate': 9.502962962962963e-06, 'epoch': 1.01}
{'loss': 0.3408, 'grad_norm': 7.226240634918213, 'learning_rate': 9.484444444444444e-06, 'epoch': 1.01}
{'loss': 0.3172, 'grad_norm': 8.998685836791992, 'learning_rate': 9.465925925925927e-06, 'epoch': 1.01}
{'loss': 0.3321, 'grad_norm': 7.751436233520508, 'learning_rate': 9.447407407407409e-06, 'epoch': 1.01}
{'loss': 0.3152, 'grad_norm': 10.260729789733887, 'learning_rate': 9.428888888888

...

{'eval_loss': 0.24130722880363464, 'eval_wer': 60.33288913276371, 'eval_runtime': 1345.3545, 'eval_samples_per_second': 6.209, 'eval_steps_per_second': 0.389, 'epoch': 9.04}


/home/ahanaf/Torch/torch2_2/lib/python3.11/site-packages/torch/utils/checkpoint.py:460: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(


{'loss': 0.0719, 'grad_norm': 5.174563884735107, 'learning_rate': 2.948888888888889e-06, 'epoch': 9.04}
{'loss': 0.0755, 'grad_norm': 6.608218669891357, 'learning_rate': 2.9303703703703705e-06, 'epoch': 9.04}
{'loss': 0.0523, 'grad_norm': 5.388758182525635, 'learning_rate': 2.9118518518518517e-06, 'epoch': 9.05}
{'loss': 0.1065, 'grad_norm': 5.887277126312256, 'learning_rate': 2.8933333333333337e-06, 'epoch': 9.05}
{'loss': 0.0886, 'grad_norm': 3.7672016620635986, 'learning_rate': 2.8748148148148154e-06, 'epoch': 9.05}
{'loss': 0.0771, 'grad_norm': 7.151106834411621, 'learning_rate': 2.8562962962962966e-06, 'epoch': 9.05}
{'loss': 0.0689, 'grad_norm': 2.7750704288482666, 'learning_rate': 2.837777777777778e-06, 'epoch': 9.05}
{'loss': 0.048, 'grad_norm': 2.4282751083374023, 'learning_rate': 2.8192592592592594e-06, 'epoch': 9.05}
{'loss': 0.0455, 'grad_norm': 3.2004592418670654, 'learning_rate': 2.800740740740741e-06, 'epoch': 9.06}
{'loss': 0.0563, 'grad_norm': 4.610203742980957, 'learn

Too many dataloader workers: 16 (max is dataset.n_shards=1). Stopping 15 dataloader workers.
Reading metadata...: 16777it [00:03, 5135.28it/s]


{'loss': 0.0796, 'grad_norm': 4.788090229034424, 'learning_rate': 2.5970370370370372e-06, 'epoch': 10.0}
{'loss': 0.0763, 'grad_norm': 6.6651930809021, 'learning_rate': 2.5785185185185184e-06, 'epoch': 10.0}
{'loss': 0.0657, 'grad_norm': 3.694990634918213, 'learning_rate': 2.56e-06, 'epoch': 10.0}
{'loss': 0.0733, 'grad_norm': 4.8404035568237305, 'learning_rate': 2.5414814814814813e-06, 'epoch': 10.01}
{'loss': 0.0668, 'grad_norm': 4.640250205993652, 'learning_rate': 2.5229629629629633e-06, 'epoch': 10.01}
{'loss': 0.0846, 'grad_norm': 4.365676403045654, 'learning_rate': 2.504444444444445e-06, 'epoch': 10.01}
{'loss': 0.0683, 'grad_norm': 4.276456356048584, 'learning_rate': 2.485925925925926e-06, 'epoch': 10.01}
{'loss': 0.0611, 'grad_norm': 5.738658905029297, 'learning_rate': 2.4674074074074073e-06, 'epoch': 10.01}
{'loss': 0.069, 'grad_norm': 5.826195240020752, 'learning_rate': 2.448888888888889e-06, 'epoch': 10.02}
{'loss': 0.0555, 'grad_norm': 3.4955544471740723, 'learning_rate': 2

In [ ]:
# kwargs = {
#     "dataset_tags": "mozilla-foundation/common_voice_11_0",
#     "dataset": "Common Voice 11.0",  # a 'pretty' name for the training dataset
#     "dataset_args": "config: bn, split: test",
#     "language": "bn",
#     "model_name": "Whisper Tiny BN - ahanaf019",  # a 'pretty' name for your model
#     "finetuned_from": "openai/whisper-tiny",
#     "tasks": "automatic-speech-recognition",
# }

In [ ]:
# trainer.push_to_hub(**kwargs)

In [ ]:
import torch

In [ ]:
torch.cuda.memory_stats()

OrderedDict([('active.all.allocated', 284789737),
             ('active.all.current', 503),
             ('active.all.freed', 284789234),
             ('active.all.peak', 775),
             ('active.large_pool.allocated', 31463846),
             ('active.large_pool.current', 57),
             ('active.large_pool.freed', 31463789),
             ('active.large_pool.peak', 106),
             ('active.small_pool.allocated', 253325891),
             ('active.small_pool.current', 446),
             ('active.small_pool.freed', 253325445),
             ('active.small_pool.peak', 698),
             ('active_bytes.all.allocated', 302894913586176),
             ('active_bytes.all.current', 465560064),
             ('active_bytes.all.freed', 302894448026112),
             ('active_bytes.all.peak', 4381111808),
             ('active_bytes.large_pool.allocated', 256030036684288),
             ('active_bytes.large_pool.current', 376891904),
             ('active_bytes.large_pool.freed', 2560296597923